In [1]:
from linear_sample_3c1 import *
import time
from collections import namedtuple
from read_images import *#不带模糊
#from read_images_flur import *#带模糊，输出左图，右图和模糊
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-

In [2]:
model_params = namedtuple("parameters",
                          'data_size,'
                           'mini_batch_size,'
                           'learning_rate,'
                           'total_epoch_num,'
                           'outputdir,'
                           'height,'
                           'width,'
                           'channel,'
                          'cutTop,'
                          'cutBottom,'
                          'cutLeft,'
                          'cutRight,'
                          'cPointRow,'
                          'cPointCol')


In [3]:
def train_feature(params, feature_in, left_ims, right_ims, tps_base, tps_matrix, linear_interpolator, mode, loss_norm_mode=True):
    print_str = 'train_tps_Step：{:4} | Reonstruction loss is {:4} | Norm loss is {:4} | Total loss is {:4}' \
                '  | feature_var_mean is{:4}'
    # with tf.Graph().as_default(), tf.device('/gpu: 0'):
    max_step = np.int32(params.total_epoch_num)
    learning_rate_init = np.float32(params.learning_rate)

    optimize_op = tf.train.AdamOptimizer(learning_rate_init)
    #optimize_op = tf.train.GradientDescentOptimizer(learning_rate_init)
    
    
    with tf.variable_scope(tf.get_variable_scope()):
        tps_base = tf.constant(tps_base, dtype=tf.float32)
        left = tf.constant(left_ims, dtype=tf.float32)
        right = tf.constant(right_ims, dtype=tf.float32)
        if mode == 'TPS':
            feature_input = tf.Variable(feature_in, dtype=tf.float32,name='contr_val', trainable=False)
            tps_weight = tf.Variable(tps_matrix, dtype=tf.float32, name='tps_val')
        elif mode == 'FEATURE':
            feature_input = tf.Variable(feature_in, dtype=tf.float32, name='contr_val')
            tps_weight = tf.Variable(tps_matrix, dtype=tf.float32, name='tps_val', trainable=False)
        else:
            feature_input = tf.Variable(feature_in, dtype=tf.float32)
            tps_weight = tf.Variable(tps_matrix, dtype=tf.float32)
        
        compensateI = tf.Variable(4.3, dtype=tf.float32, name='contr_val')###亮度补偿
        #compensateI = tf.Variable(-2.0, dtype=tf.float32, name='contr_val')###亮度补偿guijiao
        
        
        
        disp = decoder_forward1(feature_input, tps_weight, linear_interpolator.sz_params)
        #train_dd__ = np.load('groundtruth_z/disp_batch_invivo_d.npy')[0:100]
        #disp = tf.Variable(train_dd__, dtype=tf.float32)
        
        
        
        right_est = linear_interpolator.interpolate(left, disp)
        #loss_rec = compute_rec_loss(right_est, right, linear_interpolator.sz_params)
        loss_rec,compa_sum,loss_rec_sum = compute_rec_loss(right_est, right, compensateI, linear_interpolator.sz_params, 180.0)##使用膨胀做部分平滑##载体
        #loss_rec,compa_sum,loss_rec_sum = compute_rec_loss(right_est, right, compensateI, linear_interpolator.sz_params, 255.0)
        #loss_rec = compute_rec_loss3(right_est, right, left, linear_interpolator.sz_params)
        loss_wt_norm = tf.reduce_sum(tf.square(tps_weight - tps_base))
        
        #disp_i1 = tf.slice(disp, [0,  params.cutTop, params.cutLeft, 0], [-1, 256, 255, -1],
        #                 name='r_clip')
        #disp_i2 = tf.slice(disp, [0,  params.cutTop, params.cutLeft+1, 0], [-1, 256, 255, -1],
        #                 name='r_clip')
        #disp_i3 = tf.slice(disp, [0,  params.cutTop, params.cutLeft, 0], [-1, 255, 256, -1],
        #                 name='r_clip')
        #disp_i4 = tf.slice(disp, [0,  params.cutTop+1, params.cutLeft, 0], [-1, 255, 256, -1],
        #                 name='r_clip')
        
        #loss_wt_norm = tf.multiply(0.0015, tf.reduce_sum(tf.square(disp_i2 - disp_i1)), name='punishment')##横向
        #loss_wt_norm2 = tf.multiply(0.002, tf.reduce_sum(tf.square(disp_i4 - disp_i3)), name='punishment2')##纵向
        #loss = tf.add(loss_rec, loss_wt_norm, name='Total_loss')
        #loss = tf.add(loss, loss_wt_norm2, name='Total_loss')
        loss = tf.add(loss_rec, loss_wt_norm, name='Total_loss')
        
        if mode == 'FEATURE' or loss_norm_mode == False:
            #train_op = optimize_op.minimize(loss,var_list = [feature_input, compensateI])#可以指定哪些参数进行迭代
            train_op = optimize_op.minimize(loss_rec,var_list = feature_input)
            pass
        else:
            #train_op = optimize_op.minimize(loss,var_list = [feature_input, compensateI])
            train_op = optimize_op.minimize(loss_rec,var_list = feature_input)
            pass
    
    config = tf.ConfigProto(allow_soft_placement=True)
    print(1)
    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        print(1.1)
        start_time = time.time()
        loss_seq = np.zeros([max_step, 3])
        loss_rec_temp = 0.
        print(1.2)
        
        feature_before = feature_in
        feature_val = 0.
        tps_before = tps_matrix
        tps_val = 0.
        disp_val = np.zeros([params.data_size, params.height, params.width, 1])
        est_right_val = np.zeros([params.data_size, params.height, params.width, params.channel])
        step = 0
        print(2)
        res_loss = [[],[]]
        for step in range(0, max_step):
            _, feature_val, tps_val, loss_rec_val, loss_wt_norm_val, compa_sum_val,loss_rec_sum_val, loss_val, disp_val, est_right_val = sess.run(
                [train_op, feature_input, tps_weight, loss_rec, loss_wt_norm, compa_sum,loss_rec_sum, loss, disp, right_est]
            )

            
            loss_seq[step, :] = loss_rec_val, loss_wt_norm_val, loss_val
            res_loss[0].append(loss_rec_sum_val*params.data_size)##记录损失
            res_loss[1].append(compa_sum_val)##记录损失
            print(loss_rec_sum_val*params.data_size/compa_sum_val)
            if 0 == step % 10 or step + 1 == max_step:
                print('亮度补偿', sess.run(compensateI))
                #res_loss[0].append(loss_rec_val*params.data_size)##记录损失
                #res_loss[1].append(compa_sum_val)##记录损失
                tps_var_mean = np.mean(tps_val - tps_before)
                feature_var_mean = np.mean(feature_val - feature_before)
                

                print(print_str.format(step, loss_rec_val, loss_wt_norm_val, loss_val, feature_var_mean))#当前组的loss信息
                tps_before = tps_val
                feature_before = feature_val

                loss_rec_var = np.abs(loss_rec_val - loss_rec_temp)
                loss_rec_temp = loss_rec_val
                #if loss_rec_var < 1e-2 or step>=100:#######考虑使用loss，比较一下和重建误差的区别
                if step>=400:
                    break
                #loss_var_chazhi = np.abs(loss_val - loss_rec_temp)
                #loss_rec_temp = loss_val
                #if  loss_var_chazhi < 1e-2 or step>=3000:#######考虑使用loss，比较一下和重建误差的区别
                #    break
                
                
        feature = feature_val
        tps_mat = tps_val
        print('time spent {:8} '.format(time.time() - start_time))
        distance = np.mean(np.square(feature - feature_in))
        print('初始与终值的距离为:'+str(distance))
        
        return feature, tps_mat, loss_seq[0:step + 1, :], disp_val, est_right_val, distance, loss_rec_val, res_loss


def train_z(params, feature_in, tps_base0, left_ims, right_ims, result_path,iii):
    with tf.Graph().as_default(), tf.device('/gpu: 0'):
        linear_interpolator = LinearInterpolator(params)  # initialize linear interpolator
        tps_matrix = tps_base0
        loss_seq = np.empty(shape=[0, 3], dtype=np.float32)
        mode = 'FEATURE'
        print('--------------------FEATURE---------------------')
        feature_in, tps_matrix, loss_seq1, disp, est_right, distance, loss_rec_val, res_loss= train_feature(params, feature_in, left_ims, right_ims, tps_base0,
                                                                           tps_matrix, linear_interpolator, mode)
        ##loss_seq = np.concatenate([loss_seq, loss_seq1], axis=0)
        
        #np.save(os.path.join(result_path, 'disp_batch_invivo20'+str(iii)+'.npy'), disp)
        #np.save(os.path.join(result_path, 'feature_trained'+str(iii)+'.npy'), feature_in)

        return res_loss

In [ ]:
#通过83.2训练出来的结果 4*5, 训练集
if __name__ == '__main__':
    params = model_params(data_size=100,
                              mini_batch_size=100,
                              learning_rate=3e-1,
                              total_epoch_num=np.int32(10000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              #cutTop=54,
                              #cutBottom=34,
                              #cutLeft=73,
                              #cutRight=87,#######硅胶
                          
                              #cutTop=44,
                              #cutBottom=44,
                              #cutLeft=6,
                              #cutRight=154,#####载体
                          
                              cutTop=54,
                              cutBottom=34,
                              cutLeft=42,
                              cutRight=118,#####载体
                              
                              cPointRow=4,
                              cPointCol=4
                              )
    tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)
    #tps_base0 = np.loadtxt(r'A_val_real33.txt').astype(np.float32)
    #tps_base0 = np.loadtxt(r'A_val_real55.txt').astype(np.float32)
    #tps_base0 = np.load(r'./temp_A.npy').astype(np.float32)
    #ids = range(1950,2000)
    
    #feature_in = 46.4 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)###硅胶
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)###载体
    
    #feature_in = np.load('groundtruth_z/disp_batch_invivo_z.npy')[0:20]
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in0 = np.load('groundtruth_z/feature_test.npy')
    #feature_in0 = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')
    #print(feature_in0.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'####硅胶
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'######载体
    
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    loss = []
    for i in range(0,2):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        left_ims = np.array(left_ims, dtype = np.float32)
        right_ims = np.array(right_ims, dtype = np.float32)
        print(left_ims.shape)
        #feature_in = feature_in0[i*params.data_size:(i+1)*params.data_size,:,:]
        #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i) 
        #right_ims = np.array(right_ims)
        #left_ims = np.array(left_ims)
        #print(right_ims.shape)
        #for n in range(params.data_size):
        #    for ii in range(50):
         #       for jj in range(50):
         #           #right_ims[n,130+ii,160+jj,:] = [0,0,0]
         #           left_ims[n,130+ii,160+jj,:] = [0,0,0]
        #print(right_ims.shape)
        #loss.append(train_z(params, feature_in, tps_base0, left_ims, right_ims, 'output/guijiao/',i))####硅胶
        loss.append(train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i))####载体
        #print(loss)
    loss = np.array(loss)
    #print(loss.shape)
    #np.save('output/guijiao/'+ 'TPS200loss.npy', loss)#####硅胶
    #np.save('output/'+ 'TPS200loss.npy', loss)#####载体
    #np.save('output/'+ 'TPS55200loss.npy', loss)#####载体
    np.save('output/'+ 'TPS200loss400.npy', loss)#####载体400
    #np.save('output/guijiao/'+ 'TPS200loss400.npy', loss)#####硅胶400
    

(100, 288, 360, 3)
Tensor("meshgrid/mul:0", shape=(288, 360), dtype=float32, device=/device:GPU:0) Tensor("meshgrid/mul_1:0", shape=(288, 360), dtype=float32, device=/device:GPU:0)
Tensor("Reshape:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0) Tensor("Reshape_1:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Tile:0", shape=(100, 103680), dtype=float32, device=/device:GPU:0) Tensor("Tile_1:0", shape=(100, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Reshape_2:0", shape=(10368000,), dtype=float32, device=/device:GPU:0) Tensor("Reshape_3:0", shape=(10368000,), dtype=float32, device=/device:GPU:0)
--------------------FEATURE---------------------
Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(100, 40000, 1), dtype=float32)
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1
1.1
1.2
2
152.05030650300563
亮度补偿 4.3
train_tps_Step：   0 | Reonstructi

In [ ]:
GUIJIAO
67.22221374511719 33     65.77806083333333 NEW 33    65.95858383178711  44

In [4]:
#通过83.2训练出来的结果 4*5 测试集
if __name__ == '__main__':
    params = model_params(data_size=100,
                              mini_batch_size=100,
                              learning_rate=3e-1,
                              total_epoch_num=np.int32(10000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              #cutTop=40,
                              #cutBottom=0,
                              #cutLeft=0,
                              #cutRight=90,
                          
                              cutTop=44,
                              cutBottom=44,
                              cutLeft=6,
                              cutRight=154,
                              
                              cPointRow=4,
                              cPointCol=4
                              )
    tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)
    #tps_base0 = np.load(r'./groundtruth_z//tps_trained.npy').astype(np.float32)
    #ids = range(1950,2000)
    
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)
    #feature_in = np.load('groundtruth_z/disp_batch_invivo_z.npy')[0:100]
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in0 = np.load('groundtruth_z/feature_test.npy')
    #feature_in0 = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')
    #print(feature_in0.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    loss = []
    for i in range(2,3):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        left_ims = np.array(left_ims, dtype = np.float32)
        right_ims = np.array(right_ims, dtype = np.float32)
        print(left_ims.shape)
        #feature_in = feature_in0[i*params.data_size:(i+1)*params.data_size,:,:]
        #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i) 
        #right_ims = np.array(right_ims)
        #left_ims = np.array(left_ims)
        #print(right_ims.shape)
        #for n in range(params.data_size):
        #    for ii in range(50):
         #       for jj in range(50):
         #           #right_ims[n,130+ii,160+jj,:] = [0,0,0]
         #           left_ims[n,130+ii,160+jj,:] = [0,0,0]
        #print(right_ims.shape)
        loss.append(train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i))
    #loss = np.array(loss)
    #print(loss.shape)
    np.save('output/'+ 'TPSloss_test.npy', loss)

(100, 288, 360, 3)
Tensor("meshgrid/mul:0", shape=(288, 360), dtype=float32, device=/device:GPU:0) Tensor("meshgrid/mul_1:0", shape=(288, 360), dtype=float32, device=/device:GPU:0)
Tensor("Reshape:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0) Tensor("Reshape_1:0", shape=(1, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Tile:0", shape=(100, 103680), dtype=float32, device=/device:GPU:0) Tensor("Tile_1:0", shape=(100, 103680), dtype=float32, device=/device:GPU:0)
Tensor("Reshape_2:0", shape=(10368000,), dtype=float32, device=/device:GPU:0) Tensor("Reshape_3:0", shape=(10368000,), dtype=float32, device=/device:GPU:0)
--------------------FEATURE---------------------
Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(100, 40000, 1), dtype=float32)
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1
1.1
1.2
2
亮度补偿 4.3
train_tps_Step：   0 | Reonstruction loss is 18988506

In [2]:
#(98.10715+67.955116)/2
(73.89562 + 80.13996 + 66.45233 + 65.862747)/4

71.58766425

In [3]:
(77.01785+66.157516)/2

71.587683

In [4]:
#通过83.2训练出来的结果 5*6
if __name__ == '__main__':
    params = model_params(data_size=50,
                              mini_batch_size=50,
                              learning_rate=3e-2,
                              total_epoch_num=np.int32(10000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              #cutTop=40,
                              #cutBottom=0,
                              #cutLeft=0,
                              #cutRight=90,
                          
                              cutTop=32,
                              cutBottom=0,
                              cutLeft=14,
                              cutRight=90,
                              
                              cPointRow=5,
                              cPointCol=6
                              )
    tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)
    #ids = range(1950,2000)
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in0 = np.load('groundtruth_z/feature_test.npy')
    #feature_in0 = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')
    #print(feature_in0.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    loss = []
    for i in range(16,18):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        #feature_in = feature_in0[i*params.data_size:(i+1)*params.data_size,:,:]
        #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i) 
        #right_ims = np.array(right_ims)
        #left_ims = np.array(left_ims)
        #print(right_ims.shape)
        #for n in range(params.data_size):
        #    for ii in range(50):
         #       for jj in range(50):
         #           #right_ims[n,130+ii,160+jj,:] = [0,0,0]
         #           left_ims[n,130+ii,160+jj,:] = [0,0,0]
        #print(right_ims.shape)
        loss.append(train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i))
    loss = np.array(loss)
    print(loss.shape)
    #np.save('experiment_result/'+ 'loss800-900.npy', loss)

--------------------FEATURE---------------------
(50, 65536, 1)
Instructions for updating:
Use `tf.cast` instead.
(50, 256, 256, 1)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1
1.1
1.2
2
train_tps_Step：   0 | Reonstruction loss is 270.7232360839844 | Norm loss is 0.0994061678647995 | Total loss is 270.8226318359375 | loss_rec_bijiao_var_mean is 400.7828063964844 | feature_var_mean is 0.0
train_tps_Step： 100 | Reonstruction loss is 205.13589477539062 | Norm loss is 4.473794460296631 | Total loss is 209.6096954345703 | loss_rec_bijiao_var_mean is 328.8940734863281 | feature_var_mean is0.8612594604492188
train_tps_Step： 200 | Reonstruction loss is 172.3057861328125 | Norm loss is 6.958876609802246 | Total loss is 179.26466369628906 | loss_rec_bijiao_var_mean is 293.42156982421875 | feature_var_mean is0.9384168386459351
train_tps_Step： 300 | Reonstruction loss is 153.20077514648438 | Norm loss is 8.479660034179688 | Total loss is 161.68043

/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [5]:
#通过83.2训练出来的结果 5*6
if __name__ == '__main__':
    params = model_params(data_size=50,
                              mini_batch_size=50,
                              learning_rate=1e-2,
                              total_epoch_num=np.int32(10000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              #cutTop=40,
                              #cutBottom=0,
                              #cutLeft=0,
                              #cutRight=90,
                          
                              cutTop=32,
                              cutBottom=0,
                              cutLeft=14,
                              cutRight=90,
                              
                              cPointRow=5,
                              cPointCol=6
                              )
    tps_base0 = np.loadtxt(r'A_val_real.txt').astype(np.float32)
    #ids = range(1950,2000)
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in0 = np.load('groundtruth_z/feature_test.npy')
    #feature_in0 = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')
    #print(feature_in0.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    
    for i in range(0,20):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        #feature_in = feature_in0[i*params.data_size:(i+1)*params.data_size,:,:]
        #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i)  
        train_z(params, feature_in, tps_base0, left_ims, right_ims, '256_real_dataset/',i)  

--------------------FEATURE---------------------
(50, 65536, 1)
1
1.1
1.2
2
train_tps_Step：   0 | Reonstruction loss is 398.87078857421875 | Norm loss is  0.0 | Total loss is 398.87078857421875 | TPS_var_mean is  0.0 | feature_var_mean is 0.0
train_tps_Step： 100 | Reonstruction loss is 364.4249572753906 | Norm loss is  0.0 | Total loss is 364.4249572753906 | TPS_var_mean is  0.0 | feature_var_mean is-0.12306686490774155
train_tps_Step： 200 | Reonstruction loss is 341.099365234375 | Norm loss is  0.0 | Total loss is 341.099365234375 | TPS_var_mean is  0.0 | feature_var_mean is-0.11107468605041504
train_tps_Step： 300 | Reonstruction loss is 325.3622741699219 | Norm loss is  0.0 | Total loss is 325.3622741699219 | TPS_var_mean is  0.0 | feature_var_mean is-0.0828443169593811
train_tps_Step： 400 | Reonstruction loss is 315.0716247558594 | Norm loss is  0.0 | Total loss is 315.0716247558594 | TPS_var_mean is  0.0 | feature_var_mean is-0.04793496057391167
train_tps_Step： 500 | Reonstruction 

train_tps_Step：4500 | Reonstruction loss is 255.19918823242188 | Norm loss is  0.0 | Total loss is 255.19918823242188 | TPS_var_mean is  0.0 | feature_var_mean is0.0005471801850944757
train_tps_Step：4600 | Reonstruction loss is 255.1922607421875 | Norm loss is  0.0 | Total loss is 255.1922607421875 | TPS_var_mean is  0.0 | feature_var_mean is-0.0011108627077192068
train_tps_Step：4700 | Reonstruction loss is 255.18533325195312 | Norm loss is  0.0 | Total loss is 255.18533325195312 | TPS_var_mean is  0.0 | feature_var_mean is-0.0002962570288218558
train_tps_Step：4800 | Reonstruction loss is 255.18116760253906 | Norm loss is  0.0 | Total loss is 255.18116760253906 | TPS_var_mean is  0.0 | feature_var_mean is0.0005535430973395705
train_tps_Step：4900 | Reonstruction loss is 255.17640686035156 | Norm loss is  0.0 | Total loss is 255.17640686035156 | TPS_var_mean is  0.0 | feature_var_mean is0.000628301000688225
train_tps_Step：5000 | Reonstruction loss is 255.16200256347656 | Norm loss is  0.

train_tps_Step：2300 | Reonstruction loss is 260.29876708984375 | Norm loss is  0.0 | Total loss is 260.29876708984375 | TPS_var_mean is  0.0 | feature_var_mean is0.0020251593086868525
train_tps_Step：2400 | Reonstruction loss is 259.7276611328125 | Norm loss is  0.0 | Total loss is 259.7276611328125 | TPS_var_mean is  0.0 | feature_var_mean is0.0033123346511274576
train_tps_Step：2500 | Reonstruction loss is 259.3042297363281 | Norm loss is  0.0 | Total loss is 259.3042297363281 | TPS_var_mean is  0.0 | feature_var_mean is0.0014885406708344817
train_tps_Step：2600 | Reonstruction loss is 258.9038391113281 | Norm loss is  0.0 | Total loss is 258.9038391113281 | TPS_var_mean is  0.0 | feature_var_mean is0.0018756560748443007
train_tps_Step：2700 | Reonstruction loss is 258.5762634277344 | Norm loss is  0.0 | Total loss is 258.5762634277344 | TPS_var_mean is  0.0 | feature_var_mean is0.004068155772984028
train_tps_Step：2800 | Reonstruction loss is 258.27471923828125 | Norm loss is  0.0 | Tota

train_tps_Step：6800 | Reonstruction loss is 255.93312072753906 | Norm loss is  0.0 | Total loss is 255.93312072753906 | TPS_var_mean is  0.0 | feature_var_mean is0.00036095172981731594
train_tps_Step：6900 | Reonstruction loss is 255.9292755126953 | Norm loss is  0.0 | Total loss is 255.9292755126953 | TPS_var_mean is  0.0 | feature_var_mean is0.0001419804902980104
train_tps_Step：7000 | Reonstruction loss is 255.9372100830078 | Norm loss is  0.0 | Total loss is 255.9372100830078 | TPS_var_mean is  0.0 | feature_var_mean is0.001017921487800777
train_tps_Step：7100 | Reonstruction loss is 255.90975952148438 | Norm loss is  0.0 | Total loss is 255.90975952148438 | TPS_var_mean is  0.0 | feature_var_mean is0.0006610616110265255
train_tps_Step：7200 | Reonstruction loss is 255.91542053222656 | Norm loss is  0.0 | Total loss is 255.91542053222656 | TPS_var_mean is  0.0 | feature_var_mean is0.0013160323724150658
train_tps_Step：7300 | Reonstruction loss is 255.9244842529297 | Norm loss is  0.0 | 

train_tps_Step：1500 | Reonstruction loss is 197.827880859375 | Norm loss is  0.0 | Total loss is 197.827880859375 | TPS_var_mean is  0.0 | feature_var_mean is0.017762675881385803
train_tps_Step：1600 | Reonstruction loss is 197.64425659179688 | Norm loss is  0.0 | Total loss is 197.64425659179688 | TPS_var_mean is  0.0 | feature_var_mean is0.011964868754148483
train_tps_Step：1700 | Reonstruction loss is 197.60208129882812 | Norm loss is  0.0 | Total loss is 197.60208129882812 | TPS_var_mean is  0.0 | feature_var_mean is0.008650431409478188
train_tps_Step：1800 | Reonstruction loss is 197.53207397460938 | Norm loss is  0.0 | Total loss is 197.53207397460938 | TPS_var_mean is  0.0 | feature_var_mean is0.0040901233442127705
train_tps_Step：1900 | Reonstruction loss is 197.43099975585938 | Norm loss is  0.0 | Total loss is 197.43099975585938 | TPS_var_mean is  0.0 | feature_var_mean is0.004734079819172621
train_tps_Step：2000 | Reonstruction loss is 197.44105529785156 | Norm loss is  0.0 | Tot

...

train_tps_Step：2700 | Reonstruction loss is 221.06442260742188 | Norm loss is  0.0 | Total loss is 221.06442260742188 | TPS_var_mean is  0.0 | feature_var_mean is0.010525263845920563
train_tps_Step：2800 | Reonstruction loss is 220.92567443847656 | Norm loss is  0.0 | Total loss is 220.92567443847656 | TPS_var_mean is  0.0 | feature_var_mean is0.008029677905142307
train_tps_Step：2900 | Reonstruction loss is 220.83987426757812 | Norm loss is  0.0 | Total loss is 220.83987426757812 | TPS_var_mean is  0.0 | feature_var_mean is0.007154698483645916
train_tps_Step：3000 | Reonstruction loss is 220.73983764648438 | Norm loss is  0.0 | Total loss is 220.73983764648438 | TPS_var_mean is  0.0 | feature_var_mean is0.007261983118951321
train_tps_Step：3100 | Reonstruction loss is 220.62489318847656 | Norm loss is  0.0 | Total loss is 220.62489318847656 | TPS_var_mean is  0.0 | feature_var_mean is0.005820037797093391
train_tps_Step：3200 | Reonstruction loss is 220.55166625976562 | Norm loss is  0.0 | 

train_tps_Step：3300 | Reonstruction loss is 233.60072326660156 | Norm loss is  0.0 | Total loss is 233.60072326660156 | TPS_var_mean is  0.0 | feature_var_mean is0.01078442856669426
train_tps_Step：3400 | Reonstruction loss is 233.44578552246094 | Norm loss is  0.0 | Total loss is 233.44578552246094 | TPS_var_mean is  0.0 | feature_var_mean is0.01087486743927002
train_tps_Step：3500 | Reonstruction loss is 233.30296325683594 | Norm loss is  0.0 | Total loss is 233.30296325683594 | TPS_var_mean is  0.0 | feature_var_mean is0.009712799452245235
train_tps_Step：3600 | Reonstruction loss is 233.1917724609375 | Norm loss is  0.0 | Total loss is 233.1917724609375 | TPS_var_mean is  0.0 | feature_var_mean is0.0081871896982193
train_tps_Step：3700 | Reonstruction loss is 233.10458374023438 | Norm loss is  0.0 | Total loss is 233.10458374023438 | TPS_var_mean is  0.0 | feature_var_mean is0.005972549319267273
train_tps_Step：3800 | Reonstruction loss is 233.01576232910156 | Norm loss is  0.0 | Total 

train_tps_Step：1900 | Reonstruction loss is 243.01815795898438 | Norm loss is  0.0 | Total loss is 243.01815795898438 | TPS_var_mean is  0.0 | feature_var_mean is-0.1581331044435501
train_tps_Step：2000 | Reonstruction loss is 239.06442260742188 | Norm loss is  0.0 | Total loss is 239.06442260742188 | TPS_var_mean is  0.0 | feature_var_mean is-0.14145392179489136
train_tps_Step：2100 | Reonstruction loss is 235.9090576171875 | Norm loss is  0.0 | Total loss is 235.9090576171875 | TPS_var_mean is  0.0 | feature_var_mean is-0.12033996731042862
train_tps_Step：2200 | Reonstruction loss is 233.24098205566406 | Norm loss is  0.0 | Total loss is 233.24098205566406 | TPS_var_mean is  0.0 | feature_var_mean is-0.10622821748256683
train_tps_Step：2300 | Reonstruction loss is 231.31072998046875 | Norm loss is  0.0 | Total loss is 231.31072998046875 | TPS_var_mean is  0.0 | feature_var_mean is-0.08375521004199982
train_tps_Step：2400 | Reonstruction loss is 229.82666015625 | Norm loss is  0.0 | Total 

train_tps_Step：1400 | Reonstruction loss is 531.8779907226562 | Norm loss is  0.0 | Total loss is 531.8779907226562 | TPS_var_mean is  0.0 | feature_var_mean is-0.2684309482574463
train_tps_Step：1500 | Reonstruction loss is 522.4314575195312 | Norm loss is  0.0 | Total loss is 522.4314575195312 | TPS_var_mean is  0.0 | feature_var_mean is-0.24320092797279358
train_tps_Step：1600 | Reonstruction loss is 514.2958374023438 | Norm loss is  0.0 | Total loss is 514.2958374023438 | TPS_var_mean is  0.0 | feature_var_mean is-0.21592678129673004
train_tps_Step：1700 | Reonstruction loss is 507.0044860839844 | Norm loss is  0.0 | Total loss is 507.0044860839844 | TPS_var_mean is  0.0 | feature_var_mean is-0.19344879686832428
train_tps_Step：1800 | Reonstruction loss is 500.7559509277344 | Norm loss is  0.0 | Total loss is 500.7559509277344 | TPS_var_mean is  0.0 | feature_var_mean is-0.16774523258209229
train_tps_Step：1900 | Reonstruction loss is 495.3541259765625 | Norm loss is  0.0 | Total loss i

train_tps_Step：6000 | Reonstruction loss is 442.1748352050781 | Norm loss is  0.0 | Total loss is 442.1748352050781 | TPS_var_mean is  0.0 | feature_var_mean is0.006133913528174162
train_tps_Step：6100 | Reonstruction loss is 442.0736083984375 | Norm loss is  0.0 | Total loss is 442.0736083984375 | TPS_var_mean is  0.0 | feature_var_mean is0.004858817905187607
train_tps_Step：6200 | Reonstruction loss is 441.9919128417969 | Norm loss is  0.0 | Total loss is 441.9919128417969 | TPS_var_mean is  0.0 | feature_var_mean is0.0053403242491185665
train_tps_Step：6300 | Reonstruction loss is 441.9112548828125 | Norm loss is  0.0 | Total loss is 441.9112548828125 | TPS_var_mean is  0.0 | feature_var_mean is0.004311912693083286
train_tps_Step：6400 | Reonstruction loss is 441.81005859375 | Norm loss is  0.0 | Total loss is 441.81005859375 | TPS_var_mean is  0.0 | feature_var_mean is0.004007367417216301
train_tps_Step：6500 | Reonstruction loss is 441.7001953125 | Norm loss is  0.0 | Total loss is 441

In [5]:
#通过83.2训练出来的结果
if __name__ == '__main__':
    params = model_params(data_size=50,
                              mini_batch_size=50,
                              learning_rate=1e-3,
                              total_epoch_num=np.int32(10000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              cutTop=40,
                              cutBottom=0,
                              cutLeft=0,
                              cutRight=90,
                              cPointRow=6,
                              cPointCol=5
                              )
    tps_base0 = np.loadtxt(r'A_val_mat.txt').astype(np.float32)
    #ids = range(1950,2000)
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in = np.load('joint/test/feature_test.npy')
    #print(feature_in.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    
    for i in range(0,20):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i) 

--------------------FEATURE---------------------
(50, 66960, 1)
1
1.1
1.2
2
train_tps_Step：   0 | Reonstruction loss is 373.5682067871094 | Norm loss is  0.0 | Total loss is 373.5682067871094 | TPS_var_mean is  0.0 | feature_var_mean is 0.0
train_tps_Step： 100 | Reonstruction loss is 370.39306640625 | Norm loss is  0.0 | Total loss is 370.39306640625 | TPS_var_mean is  0.0 | feature_var_mean is0.0037585499230772257
train_tps_Step： 200 | Reonstruction loss is 368.1632385253906 | Norm loss is  0.0 | Total loss is 368.1632385253906 | TPS_var_mean is  0.0 | feature_var_mean is0.004531539976596832
train_tps_Step： 300 | Reonstruction loss is 364.8094787597656 | Norm loss is  0.0 | Total loss is 364.8094787597656 | TPS_var_mean is  0.0 | feature_var_mean is-0.00987656693905592
train_tps_Step： 400 | Reonstruction loss is 359.97705078125 | Norm loss is  0.0 | Total loss is 359.97705078125 | TPS_var_mean is  0.0 | feature_var_mean is-0.024697719141840935
train_tps_Step： 500 | Reonstruction loss 

train_tps_Step：4500 | Reonstruction loss is 281.99261474609375 | Norm loss is  0.0 | Total loss is 281.99261474609375 | TPS_var_mean is  0.0 | feature_var_mean is-0.004895029589533806
train_tps_Step：4600 | Reonstruction loss is 281.205078125 | Norm loss is  0.0 | Total loss is 281.205078125 | TPS_var_mean is  0.0 | feature_var_mean is-0.005736831575632095
train_tps_Step：4700 | Reonstruction loss is 280.43548583984375 | Norm loss is  0.0 | Total loss is 280.43548583984375 | TPS_var_mean is  0.0 | feature_var_mean is-0.005384994670748711
train_tps_Step：4800 | Reonstruction loss is 279.6605224609375 | Norm loss is  0.0 | Total loss is 279.6605224609375 | TPS_var_mean is  0.0 | feature_var_mean is-0.005001647863537073
train_tps_Step：4900 | Reonstruction loss is 278.97283935546875 | Norm loss is  0.0 | Total loss is 278.97283935546875 | TPS_var_mean is  0.0 | feature_var_mean is-0.005523015279322863
train_tps_Step：5000 | Reonstruction loss is 278.2992248535156 | Norm loss is  0.0 | Total lo

train_tps_Step：9100 | Reonstruction loss is 267.55096435546875 | Norm loss is  0.0 | Total loss is 267.55096435546875 | TPS_var_mean is  0.0 | feature_var_mean is0.006738204974681139
train_tps_Step：9200 | Reonstruction loss is 267.5171813964844 | Norm loss is  0.0 | Total loss is 267.5171813964844 | TPS_var_mean is  0.0 | feature_var_mean is0.007012374699115753
train_tps_Step：9300 | Reonstruction loss is 267.4320068359375 | Norm loss is  0.0 | Total loss is 267.4320068359375 | TPS_var_mean is  0.0 | feature_var_mean is0.007890182547271252
train_tps_Step：9400 | Reonstruction loss is 267.2936096191406 | Norm loss is  0.0 | Total loss is 267.2936096191406 | TPS_var_mean is  0.0 | feature_var_mean is0.008129272609949112
train_tps_Step：9500 | Reonstruction loss is 267.22119140625 | Norm loss is  0.0 | Total loss is 267.22119140625 | TPS_var_mean is  0.0 | feature_var_mean is0.00841897539794445
train_tps_Step：9600 | Reonstruction loss is 267.073486328125 | Norm loss is  0.0 | Total loss is 2

train_tps_Step：3500 | Reonstruction loss is 328.1747131347656 | Norm loss is  0.0 | Total loss is 328.1747131347656 | TPS_var_mean is  0.0 | feature_var_mean is0.006827758625149727
train_tps_Step：3600 | Reonstruction loss is 326.7451171875 | Norm loss is  0.0 | Total loss is 326.7451171875 | TPS_var_mean is  0.0 | feature_var_mean is0.006590317003428936
train_tps_Step：3700 | Reonstruction loss is 325.2169189453125 | Norm loss is  0.0 | Total loss is 325.2169189453125 | TPS_var_mean is  0.0 | feature_var_mean is0.006190165411680937
train_tps_Step：3800 | Reonstruction loss is 323.7677917480469 | Norm loss is  0.0 | Total loss is 323.7677917480469 | TPS_var_mean is  0.0 | feature_var_mean is0.005923868622630835
train_tps_Step：3900 | Reonstruction loss is 322.4053649902344 | Norm loss is  0.0 | Total loss is 322.4053649902344 | TPS_var_mean is  0.0 | feature_var_mean is0.0065571339800953865
train_tps_Step：4000 | Reonstruction loss is 321.07440185546875 | Norm loss is  0.0 | Total loss is 3

train_tps_Step：8100 | Reonstruction loss is 293.3072509765625 | Norm loss is  0.0 | Total loss is 293.3072509765625 | TPS_var_mean is  0.0 | feature_var_mean is0.010252350009977818
train_tps_Step：8200 | Reonstruction loss is 293.11181640625 | Norm loss is  0.0 | Total loss is 293.11181640625 | TPS_var_mean is  0.0 | feature_var_mean is0.010677576996386051
train_tps_Step：8300 | Reonstruction loss is 292.79766845703125 | Norm loss is  0.0 | Total loss is 292.79766845703125 | TPS_var_mean is  0.0 | feature_var_mean is0.011334777809679508
train_tps_Step：8400 | Reonstruction loss is 292.53302001953125 | Norm loss is  0.0 | Total loss is 292.53302001953125 | TPS_var_mean is  0.0 | feature_var_mean is0.010760609991848469
train_tps_Step：8500 | Reonstruction loss is 292.2251281738281 | Norm loss is  0.0 | Total loss is 292.2251281738281 | TPS_var_mean is  0.0 | feature_var_mean is0.01053891982883215
train_tps_Step：8600 | Reonstruction loss is 292.0299377441406 | Norm loss is  0.0 | Total loss i

...

train_tps_Step： 700 | Reonstruction loss is 596.29443359375 | Norm loss is  0.0 | Total loss is 596.29443359375 | TPS_var_mean is  0.0 | feature_var_mean is-0.04731026664376259
train_tps_Step： 800 | Reonstruction loss is 592.688232421875 | Norm loss is  0.0 | Total loss is 592.688232421875 | TPS_var_mean is  0.0 | feature_var_mean is-0.04441772401332855
train_tps_Step： 900 | Reonstruction loss is 589.5645141601562 | Norm loss is  0.0 | Total loss is 589.5645141601562 | TPS_var_mean is  0.0 | feature_var_mean is-0.03975686803460121
train_tps_Step：1000 | Reonstruction loss is 586.883544921875 | Norm loss is  0.0 | Total loss is 586.883544921875 | TPS_var_mean is  0.0 | feature_var_mean is-0.034177422523498535
train_tps_Step：1100 | Reonstruction loss is 584.3720092773438 | Norm loss is  0.0 | Total loss is 584.3720092773438 | TPS_var_mean is  0.0 | feature_var_mean is-0.029604416340589523
train_tps_Step：1200 | Reonstruction loss is 581.3763427734375 | Norm loss is  0.0 | Total loss is 581

train_tps_Step：5300 | Reonstruction loss is 438.41094970703125 | Norm loss is  0.0 | Total loss is 438.41094970703125 | TPS_var_mean is  0.0 | feature_var_mean is-0.03216389939188957
train_tps_Step：5400 | Reonstruction loss is 435.135009765625 | Norm loss is  0.0 | Total loss is 435.135009765625 | TPS_var_mean is  0.0 | feature_var_mean is-0.032467324286699295
train_tps_Step：5500 | Reonstruction loss is 431.9927062988281 | Norm loss is  0.0 | Total loss is 431.9927062988281 | TPS_var_mean is  0.0 | feature_var_mean is-0.03220485523343086
train_tps_Step：5600 | Reonstruction loss is 429.0216369628906 | Norm loss is  0.0 | Total loss is 429.0216369628906 | TPS_var_mean is  0.0 | feature_var_mean is-0.03261905908584595
train_tps_Step：5700 | Reonstruction loss is 426.18060302734375 | Norm loss is  0.0 | Total loss is 426.18060302734375 | TPS_var_mean is  0.0 | feature_var_mean is-0.03201749175786972
train_tps_Step：5800 | Reonstruction loss is 423.3414306640625 | Norm loss is  0.0 | Total lo

train_tps_Step：9900 | Reonstruction loss is 321.7276306152344 | Norm loss is  0.0 | Total loss is 321.7276306152344 | TPS_var_mean is  0.0 | feature_var_mean is-0.022199681028723717
train_tps_Step：9999 | Reonstruction loss is 319.845458984375 | Norm loss is  0.0 | Total loss is 319.845458984375 | TPS_var_mean is  0.0 | feature_var_mean is-0.02175239473581314
time spent 645.7769258022308 
初始与终值的距离为:50.003635
--------------------FEATURE---------------------
(50, 66960, 1)
1
1.1
1.2
2
train_tps_Step：   0 | Reonstruction loss is 1017.2183837890625 | Norm loss is  0.0 | Total loss is 1017.2183837890625 | TPS_var_mean is  0.0 | feature_var_mean is 0.0
train_tps_Step： 100 | Reonstruction loss is 1011.8667602539062 | Norm loss is  0.0 | Total loss is 1011.8667602539062 | TPS_var_mean is  0.0 | feature_var_mean is-0.015818653628230095
train_tps_Step： 200 | Reonstruction loss is 1009.8163452148438 | Norm loss is  0.0 | Total loss is 1009.8163452148438 | TPS_var_mean is  0.0 | feature_var_mean is

train_tps_Step：4300 | Reonstruction loss is 802.50732421875 | Norm loss is  0.0 | Total loss is 802.50732421875 | TPS_var_mean is  0.0 | feature_var_mean is-0.04096467047929764
train_tps_Step：4400 | Reonstruction loss is 797.5166015625 | Norm loss is  0.0 | Total loss is 797.5166015625 | TPS_var_mean is  0.0 | feature_var_mean is-0.04222486540675163
train_tps_Step：4500 | Reonstruction loss is 792.6820068359375 | Norm loss is  0.0 | Total loss is 792.6820068359375 | TPS_var_mean is  0.0 | feature_var_mean is-0.04139171913266182
train_tps_Step：4600 | Reonstruction loss is 787.9003295898438 | Norm loss is  0.0 | Total loss is 787.9003295898438 | TPS_var_mean is  0.0 | feature_var_mean is-0.041348282247781754
train_tps_Step：4700 | Reonstruction loss is 783.0455322265625 | Norm loss is  0.0 | Total loss is 783.0455322265625 | TPS_var_mean is  0.0 | feature_var_mean is-0.041210323572158813
train_tps_Step：4800 | Reonstruction loss is 778.6839599609375 | Norm loss is  0.0 | Total loss is 778.6

train_tps_Step：8900 | Reonstruction loss is 616.9984741210938 | Norm loss is  0.0 | Total loss is 616.9984741210938 | TPS_var_mean is  0.0 | feature_var_mean is-0.03143345192074776
train_tps_Step：9000 | Reonstruction loss is 613.6751098632812 | Norm loss is  0.0 | Total loss is 613.6751098632812 | TPS_var_mean is  0.0 | feature_var_mean is-0.030113590881228447
train_tps_Step：9100 | Reonstruction loss is 610.4810180664062 | Norm loss is  0.0 | Total loss is 610.4810180664062 | TPS_var_mean is  0.0 | feature_var_mean is-0.028986679390072823
train_tps_Step：9200 | Reonstruction loss is 607.3434448242188 | Norm loss is  0.0 | Total loss is 607.3434448242188 | TPS_var_mean is  0.0 | feature_var_mean is-0.028049418702721596
train_tps_Step：9300 | Reonstruction loss is 604.1417236328125 | Norm loss is  0.0 | Total loss is 604.1417236328125 | TPS_var_mean is  0.0 | feature_var_mean is-0.028876932337880135
train_tps_Step：9400 | Reonstruction loss is 600.95556640625 | Norm loss is  0.0 | Total los

In [ ]:
if __name__ == '__main__':
    params = model_params(data_size=1,
                              mini_batch_size=1,
                              learning_rate=5e-2,
                              total_epoch_num=np.int32(10000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              cutTop=40,
                              cutBottom=0,
                              cutLeft=0,
                              cutRight=90,
                              cPointRow=5,
                              cPointCol=6
                              )
    tps_base0 = np.loadtxt(r'A_val_mat.txt').astype(np.float32)
    #ids = range(1950,2000)
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[100:150,:,:]
    
    feature_in_all = np.load('groundtruth_z/feature_test.npy')
    print(feature_in_all.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    distance = []
    for i in range(100,150):
        print(i)
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        feature_in = feature_in_all[i*params.data_size-100:(i+1)*params.data_size-100]
        #if i>850:
        #    feature_in = np.load('groundtruth_z/'+'feature_trained_laststep.npy')
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        distance.append(train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i))
    distance = np.array(distance)
    np.save(r'groundtruth_z/distance.npy', distance)

(50, 30, 1)
100
--------------------FEATURE---------------------
(1, 66960, 1)
1
1.1
1.2
2
train_tps_Step：   0 | Reonstruction loss is 347.67864990234375 | Norm loss is  0.0 | Total loss is 347.67864990234375 | TPS_var_mean is  0.0 | feature_var_mean is 0.0
train_tps_Step： 100 | Reonstruction loss is 311.3967590332031 | Norm loss is  0.0 | Total loss is 311.3967590332031 | TPS_var_mean is  0.0 | feature_var_mean is0.6674445867538452
train_tps_Step： 200 | Reonstruction loss is 280.2301330566406 | Norm loss is  0.0 | Total loss is 280.2301330566406 | TPS_var_mean is  0.0 | feature_var_mean is0.5680941343307495
train_tps_Step： 300 | Reonstruction loss is 250.35726928710938 | Norm loss is  0.0 | Total loss is 250.35726928710938 | TPS_var_mean is  0.0 | feature_var_mean is0.4329826235771179
train_tps_Step： 400 | Reonstruction loss is 227.15386962890625 | Norm loss is  0.0 | Total loss is 227.15386962890625 | TPS_var_mean is  0.0 | feature_var_mean is0.4299565553665161
train_tps_Step： 500 | 

train_tps_Step： 200 | Reonstruction loss is 278.9613342285156 | Norm loss is  0.0 | Total loss is 278.9613342285156 | TPS_var_mean is  0.0 | feature_var_mean is0.5837687253952026
train_tps_Step： 300 | Reonstruction loss is 249.75926208496094 | Norm loss is  0.0 | Total loss is 249.75926208496094 | TPS_var_mean is  0.0 | feature_var_mean is0.4731025695800781
train_tps_Step： 400 | Reonstruction loss is 226.57376098632812 | Norm loss is  0.0 | Total loss is 226.57376098632812 | TPS_var_mean is  0.0 | feature_var_mean is0.4601295590400696
train_tps_Step： 500 | Reonstruction loss is 211.1797332763672 | Norm loss is  0.0 | Total loss is 211.1797332763672 | TPS_var_mean is  0.0 | feature_var_mean is0.38704654574394226
train_tps_Step： 600 | Reonstruction loss is 202.43289184570312 | Norm loss is  0.0 | Total loss is 202.43289184570312 | TPS_var_mean is  0.0 | feature_var_mean is0.24928410351276398
train_tps_Step： 700 | Reonstruction loss is 197.33106994628906 | Norm loss is  0.0 | Total loss i

In [4]:
#通过83.2训练出来的结果
if __name__ == '__main__':
    params = model_params(data_size=50,
                              mini_batch_size=50,
                              learning_rate=1e-2,
                              total_epoch_num=np.int32(10000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              cutTop=40,
                              cutBottom=0,
                              cutLeft=0,
                              cutRight=90,
                              cPointRow=6,
                              cPointCol=5
                              )
    tps_base0 = np.loadtxt(r'A_val_mat.txt').astype(np.float32)
    #ids = range(1950,2000)
    feature_in = 83.2 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)
    
    #feature_in = 83.2 * np.random.normal(size=(params.data_size, params.cPointRow * params.cPointCol, 1))
    
    #feature_in = np.load('groundtruth_z/'+'feature_trained_final'+'.npy')[0:50,:,:]
    #feature_in = np.load('joint/test/feature_test.npy')
    #print(feature_in.shape)

    #source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/real_heart/invivo1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    
    for i in range(0,20):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i)  

--------------------FEATURE---------------------
1
1.1
1.2
2
train_tps_Step：   0 | Reonstruction loss is 373.5682067871094 | Norm loss is  0.0 | Total loss is 373.5682067871094 | TPS_var_mean is  0.0 | feature_var_mean is 0.0
train_tps_Step： 100 | Reonstruction loss is 344.9300231933594 | Norm loss is  0.0 | Total loss is 344.9300231933594 | TPS_var_mean is  0.0 | feature_var_mean is-0.1409832090139389
train_tps_Step： 200 | Reonstruction loss is 320.1871337890625 | Norm loss is  0.0 | Total loss is 320.1871337890625 | TPS_var_mean is  0.0 | feature_var_mean is-0.17554478347301483
train_tps_Step： 300 | Reonstruction loss is 299.2135009765625 | Norm loss is  0.0 | Total loss is 299.2135009765625 | TPS_var_mean is  0.0 | feature_var_mean is-0.11882731318473816
train_tps_Step： 400 | Reonstruction loss is 286.5664978027344 | Norm loss is  0.0 | Total loss is 286.5664978027344 | TPS_var_mean is  0.0 | feature_var_mean is-0.1009678766131401
train_tps_Step： 500 | Reonstruction loss is 278.8575

train_tps_Step：4600 | Reonstruction loss is 252.1990203857422 | Norm loss is  0.0 | Total loss is 252.1990203857422 | TPS_var_mean is  0.0 | feature_var_mean is0.023808296769857407
train_tps_Step：4700 | Reonstruction loss is 252.0822296142578 | Norm loss is  0.0 | Total loss is 252.0822296142578 | TPS_var_mean is  0.0 | feature_var_mean is0.02036135084927082
train_tps_Step：4800 | Reonstruction loss is 251.98907470703125 | Norm loss is  0.0 | Total loss is 251.98907470703125 | TPS_var_mean is  0.0 | feature_var_mean is0.01870478130877018
train_tps_Step：4900 | Reonstruction loss is 251.91671752929688 | Norm loss is  0.0 | Total loss is 251.91671752929688 | TPS_var_mean is  0.0 | feature_var_mean is0.016277911141514778
train_tps_Step：5000 | Reonstruction loss is 251.81295776367188 | Norm loss is  0.0 | Total loss is 251.81295776367188 | TPS_var_mean is  0.0 | feature_var_mean is0.018364954739809036
train_tps_Step：5100 | Reonstruction loss is 251.72987365722656 | Norm loss is  0.0 | Total 

train_tps_Step：2900 | Reonstruction loss is 270.689453125 | Norm loss is  0.0 | Total loss is 270.689453125 | TPS_var_mean is  0.0 | feature_var_mean is0.06050367280840874
train_tps_Step：3000 | Reonstruction loss is 269.9381103515625 | Norm loss is  0.0 | Total loss is 269.9381103515625 | TPS_var_mean is  0.0 | feature_var_mean is0.06099085882306099
train_tps_Step：3100 | Reonstruction loss is 269.1991882324219 | Norm loss is  0.0 | Total loss is 269.1991882324219 | TPS_var_mean is  0.0 | feature_var_mean is0.05715328827500343
train_tps_Step：3200 | Reonstruction loss is 268.6659851074219 | Norm loss is  0.0 | Total loss is 268.6659851074219 | TPS_var_mean is  0.0 | feature_var_mean is0.04922877624630928
train_tps_Step：3300 | Reonstruction loss is 268.1931457519531 | Norm loss is  0.0 | Total loss is 268.1931457519531 | TPS_var_mean is  0.0 | feature_var_mean is0.04759731888771057
train_tps_Step：3400 | Reonstruction loss is 267.73699951171875 | Norm loss is  0.0 | Total loss is 267.73699

train_tps_Step：7500 | Reonstruction loss is 259.9755859375 | Norm loss is  0.0 | Total loss is 259.9755859375 | TPS_var_mean is  0.0 | feature_var_mean is0.009911730885505676
train_tps_Step：7600 | Reonstruction loss is 259.9174499511719 | Norm loss is  0.0 | Total loss is 259.9174499511719 | TPS_var_mean is  0.0 | feature_var_mean is0.007930804044008255
train_tps_Step：7700 | Reonstruction loss is 259.8851623535156 | Norm loss is  0.0 | Total loss is 259.8851623535156 | TPS_var_mean is  0.0 | feature_var_mean is0.012011311948299408
train_tps_Step：7800 | Reonstruction loss is 259.8660583496094 | Norm loss is  0.0 | Total loss is 259.8660583496094 | TPS_var_mean is  0.0 | feature_var_mean is0.012387610971927643
train_tps_Step：7900 | Reonstruction loss is 259.8462829589844 | Norm loss is  0.0 | Total loss is 259.8462829589844 | TPS_var_mean is  0.0 | feature_var_mean is0.014082695357501507
train_tps_Step：8000 | Reonstruction loss is 259.8290710449219 | Norm loss is  0.0 | Total loss is 259

train_tps_Step： 600 | Reonstruction loss is 225.2708740234375 | Norm loss is  0.0 | Total loss is 225.2708740234375 | TPS_var_mean is  0.0 | feature_var_mean is0.08030147850513458
train_tps_Step： 700 | Reonstruction loss is 220.84719848632812 | Norm loss is  0.0 | Total loss is 220.84719848632812 | TPS_var_mean is  0.0 | feature_var_mean is0.09271000325679779
train_tps_Step： 800 | Reonstruction loss is 218.18093872070312 | Norm loss is  0.0 | Total loss is 218.18093872070312 | TPS_var_mean is  0.0 | feature_var_mean is0.09092600643634796
train_tps_Step： 900 | Reonstruction loss is 216.3803253173828 | Norm loss is  0.0 | Total loss is 216.3803253173828 | TPS_var_mean is  0.0 | feature_var_mean is0.08251508325338364
train_tps_Step：1000 | Reonstruction loss is 215.04351806640625 | Norm loss is  0.0 | Total loss is 215.04351806640625 | TPS_var_mean is  0.0 | feature_var_mean is0.07490862905979156
train_tps_Step：1100 | Reonstruction loss is 214.07115173339844 | Norm loss is  0.0 | Total los

...

train_tps_Step： 700 | Reonstruction loss is 237.9752960205078 | Norm loss is  0.0 | Total loss is 237.9752960205078 | TPS_var_mean is  0.0 | feature_var_mean is-0.0018600820330902934
train_tps_Step： 800 | Reonstruction loss is 236.6452178955078 | Norm loss is  0.0 | Total loss is 236.6452178955078 | TPS_var_mean is  0.0 | feature_var_mean is0.020261581987142563
train_tps_Step： 900 | Reonstruction loss is 235.64871215820312 | Norm loss is  0.0 | Total loss is 235.64871215820312 | TPS_var_mean is  0.0 | feature_var_mean is0.035012394189834595
train_tps_Step：1000 | Reonstruction loss is 234.92027282714844 | Norm loss is  0.0 | Total loss is 234.92027282714844 | TPS_var_mean is  0.0 | feature_var_mean is0.04399038851261139
train_tps_Step：1100 | Reonstruction loss is 234.5038299560547 | Norm loss is  0.0 | Total loss is 234.5038299560547 | TPS_var_mean is  0.0 | feature_var_mean is0.03817169740796089
train_tps_Step：1200 | Reonstruction loss is 234.26959228515625 | Norm loss is  0.0 | Total 

train_tps_Step： 200 | Reonstruction loss is 383.1323547363281 | Norm loss is  0.0 | Total loss is 383.1323547363281 | TPS_var_mean is  0.0 | feature_var_mean is0.23431330919265747
train_tps_Step： 300 | Reonstruction loss is 357.72027587890625 | Norm loss is  0.0 | Total loss is 357.72027587890625 | TPS_var_mean is  0.0 | feature_var_mean is0.2526581585407257
train_tps_Step： 400 | Reonstruction loss is 335.5284729003906 | Norm loss is  0.0 | Total loss is 335.5284729003906 | TPS_var_mean is  0.0 | feature_var_mean is0.25053301453590393
train_tps_Step： 500 | Reonstruction loss is 315.9315490722656 | Norm loss is  0.0 | Total loss is 315.9315490722656 | TPS_var_mean is  0.0 | feature_var_mean is0.2513176202774048
train_tps_Step： 600 | Reonstruction loss is 299.31781005859375 | Norm loss is  0.0 | Total loss is 299.31781005859375 | TPS_var_mean is  0.0 | feature_var_mean is0.25055742263793945
train_tps_Step： 700 | Reonstruction loss is 285.5001220703125 | Norm loss is  0.0 | Total loss is 

train_tps_Step：1600 | Reonstruction loss is 245.5117950439453 | Norm loss is  0.0 | Total loss is 245.5117950439453 | TPS_var_mean is  0.0 | feature_var_mean is-0.12212473899126053
train_tps_Step：1700 | Reonstruction loss is 242.94293212890625 | Norm loss is  0.0 | Total loss is 242.94293212890625 | TPS_var_mean is  0.0 | feature_var_mean is-0.09441974014043808
train_tps_Step：1800 | Reonstruction loss is 241.1283416748047 | Norm loss is  0.0 | Total loss is 241.1283416748047 | TPS_var_mean is  0.0 | feature_var_mean is-0.07407282292842865
train_tps_Step：1900 | Reonstruction loss is 239.55709838867188 | Norm loss is  0.0 | Total loss is 239.55709838867188 | TPS_var_mean is  0.0 | feature_var_mean is-0.06352271884679794
train_tps_Step：2000 | Reonstruction loss is 238.37619018554688 | Norm loss is  0.0 | Total loss is 238.37619018554688 | TPS_var_mean is  0.0 | feature_var_mean is-0.04887143895030022
train_tps_Step：2100 | Reonstruction loss is 237.53244018554688 | Norm loss is  0.0 | Tota

train_tps_Step：1400 | Reonstruction loss is 510.8135986328125 | Norm loss is  0.0 | Total loss is 510.8135986328125 | TPS_var_mean is  0.0 | feature_var_mean is-0.13706697523593903
train_tps_Step：1500 | Reonstruction loss is 503.86932373046875 | Norm loss is  0.0 | Total loss is 503.86932373046875 | TPS_var_mean is  0.0 | feature_var_mean is-0.1172347292304039
train_tps_Step：1600 | Reonstruction loss is 497.95172119140625 | Norm loss is  0.0 | Total loss is 497.95172119140625 | TPS_var_mean is  0.0 | feature_var_mean is-0.0954727753996849
train_tps_Step：1700 | Reonstruction loss is 492.9483947753906 | Norm loss is  0.0 | Total loss is 492.9483947753906 | TPS_var_mean is  0.0 | feature_var_mean is-0.08498569577932358
train_tps_Step：1800 | Reonstruction loss is 489.2665710449219 | Norm loss is  0.0 | Total loss is 489.2665710449219 | TPS_var_mean is  0.0 | feature_var_mean is-0.07307200133800507
train_tps_Step：1900 | Reonstruction loss is 486.07208251953125 | Norm loss is  0.0 | Total lo

train_tps_Step：6000 | Reonstruction loss is 438.9292297363281 | Norm loss is  0.0 | Total loss is 438.9292297363281 | TPS_var_mean is  0.0 | feature_var_mean is0.027773864567279816
train_tps_Step：6100 | Reonstruction loss is 438.12762451171875 | Norm loss is  0.0 | Total loss is 438.12762451171875 | TPS_var_mean is  0.0 | feature_var_mean is0.03406020998954773
train_tps_Step：6200 | Reonstruction loss is 437.36248779296875 | Norm loss is  0.0 | Total loss is 437.36248779296875 | TPS_var_mean is  0.0 | feature_var_mean is0.030091580003499985
train_tps_Step：6300 | Reonstruction loss is 436.6891174316406 | Norm loss is  0.0 | Total loss is 436.6891174316406 | TPS_var_mean is  0.0 | feature_var_mean is0.032536111772060394
train_tps_Step：6400 | Reonstruction loss is 435.965087890625 | Norm loss is  0.0 | Total loss is 435.965087890625 | TPS_var_mean is  0.0 | feature_var_mean is0.029135722666978836
train_tps_Step：6500 | Reonstruction loss is 435.219482421875 | Norm loss is  0.0 | Total loss 

In [5]:
#4分钟就可以训练出来
#使用正态分布83.2收敛不了

In [4]:
if __name__ == '__main__':
    params = model_params(data_size=50,
                              mini_batch_size=50,
                              learning_rate=1e-3,
                              total_epoch_num=np.int32(20000),
                              outputdir=r'output',
                              height=288,
                              width=360,
                              channel=3,
                              cutTop=20,
                              cutBottom=0,
                              cutLeft=0,
                              cutRight=50,
                              cPointRow=4,
                              cPointCol=5
                              )
    tps_base0 = np.loadtxt(r'TPS_matrix_constant.txt').astype(np.float32)
    ids = range(params.data_size)
    feature_in = 48 * np.ones([params.data_size, params.cPointRow * params.cPointCol, 1], dtype=np.float32)

    source_img_path = '/home/ubuntu/jupyter_workspace/dataset/guijiaoheart/phantom1_rect/'
    #left_ims, right_ims = read_stereo_images(source_img_path, ids)
    #train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',1)
    
    
    for i in range(0,20):
        ids  = range(i*params.data_size,(i+1)*params.data_size)
        left_ims, right_ims = read_stereo_images(source_img_path, ids)
        train_z(params, feature_in, tps_base0, left_ims, right_ims, 'groundtruth_z/',i)   
        

--------------------FEATURE---------------------
1
1.1
1.2
2
train_tps_Step：   0 | Reonstruction loss is 159.80999755859375 | Norm loss is  0.0 | Total loss is 159.80999755859375 | TPS_var_mean is  0.0 | feature_var_mean is 0.0
train_tps_Step： 100 | Reonstruction loss is 158.45225524902344 | Norm loss is  0.0 | Total loss is 158.45225524902344 | TPS_var_mean is  0.0 | feature_var_mean is-0.03250417113304138
train_tps_Step： 200 | Reonstruction loss is 154.56341552734375 | Norm loss is  0.0 | Total loss is 154.56341552734375 | TPS_var_mean is  0.0 | feature_var_mean is-0.03207458555698395
train_tps_Step： 300 | Reonstruction loss is 150.89649963378906 | Norm loss is  0.0 | Total loss is 150.89649963378906 | TPS_var_mean is  0.0 | feature_var_mean is-0.03264976665377617
train_tps_Step： 400 | Reonstruction loss is 147.7632598876953 | Norm loss is  0.0 | Total loss is 147.7632598876953 | TPS_var_mean is  0.0 | feature_var_mean is-0.032304152846336365
train_tps_Step： 500 | Reonstruction loss 

train_tps_Step：4500 | Reonstruction loss is 92.46560668945312 | Norm loss is  0.0 | Total loss is 92.46560668945312 | TPS_var_mean is  0.0 | feature_var_mean is-0.03929917886853218
train_tps_Step：4600 | Reonstruction loss is 91.85913848876953 | Norm loss is  0.0 | Total loss is 91.85913848876953 | TPS_var_mean is  0.0 | feature_var_mean is-0.03980233520269394
train_tps_Step：4700 | Reonstruction loss is 91.29657745361328 | Norm loss is  0.0 | Total loss is 91.29657745361328 | TPS_var_mean is  0.0 | feature_var_mean is-0.03953511640429497
train_tps_Step：4800 | Reonstruction loss is 90.75508880615234 | Norm loss is  0.0 | Total loss is 90.75508880615234 | TPS_var_mean is  0.0 | feature_var_mean is-0.03894615173339844
train_tps_Step：4900 | Reonstruction loss is 90.18437957763672 | Norm loss is  0.0 | Total loss is 90.18437957763672 | TPS_var_mean is  0.0 | feature_var_mean is-0.038566574454307556
train_tps_Step：5000 | Reonstruction loss is 89.657958984375 | Norm loss is  0.0 | Total loss i

train_tps_Step：9100 | Reonstruction loss is 79.29144287109375 | Norm loss is  0.0 | Total loss is 79.29144287109375 | TPS_var_mean is  0.0 | feature_var_mean is-0.03251555934548378
train_tps_Step：9200 | Reonstruction loss is 79.18673706054688 | Norm loss is  0.0 | Total loss is 79.18673706054688 | TPS_var_mean is  0.0 | feature_var_mean is-0.031701453030109406
train_tps_Step：9300 | Reonstruction loss is 79.05694580078125 | Norm loss is  0.0 | Total loss is 79.05694580078125 | TPS_var_mean is  0.0 | feature_var_mean is-0.031386878341436386
train_tps_Step：9400 | Reonstruction loss is 78.95210266113281 | Norm loss is  0.0 | Total loss is 78.95210266113281 | TPS_var_mean is  0.0 | feature_var_mean is-0.030722729861736298
train_tps_Step：9500 | Reonstruction loss is 78.8423080444336 | Norm loss is  0.0 | Total loss is 78.8423080444336 | TPS_var_mean is  0.0 | feature_var_mean is-0.030143775045871735
train_tps_Step：9600 | Reonstruction loss is 78.74201965332031 | Norm loss is  0.0 | Total los

train_tps_Step：13700 | Reonstruction loss is 76.7154769897461 | Norm loss is  0.0 | Total loss is 76.7154769897461 | TPS_var_mean is  0.0 | feature_var_mean is-0.01994617097079754
train_tps_Step：13800 | Reonstruction loss is 76.70113372802734 | Norm loss is  0.0 | Total loss is 76.70113372802734 | TPS_var_mean is  0.0 | feature_var_mean is-0.02005719766020775
train_tps_Step：13900 | Reonstruction loss is 76.6858139038086 | Norm loss is  0.0 | Total loss is 76.6858139038086 | TPS_var_mean is  0.0 | feature_var_mean is-0.020057473331689835
train_tps_Step：14000 | Reonstruction loss is 76.66905975341797 | Norm loss is  0.0 | Total loss is 76.66905975341797 | TPS_var_mean is  0.0 | feature_var_mean is-0.019703572615981102
train_tps_Step：14100 | Reonstruction loss is 76.65550231933594 | Norm loss is  0.0 | Total loss is 76.65550231933594 | TPS_var_mean is  0.0 | feature_var_mean is-0.0193178690969944
train_tps_Step：14200 | Reonstruction loss is 76.64505004882812 | Norm loss is  0.0 | Total lo

train_tps_Step：3000 | Reonstruction loss is 105.00434875488281 | Norm loss is  0.0 | Total loss is 105.00434875488281 | TPS_var_mean is  0.0 | feature_var_mean is-0.03504454344511032
train_tps_Step：3100 | Reonstruction loss is 103.86824035644531 | Norm loss is  0.0 | Total loss is 103.86824035644531 | TPS_var_mean is  0.0 | feature_var_mean is-0.0358705073595047
train_tps_Step：3200 | Reonstruction loss is 102.76536560058594 | Norm loss is  0.0 | Total loss is 102.76536560058594 | TPS_var_mean is  0.0 | feature_var_mean is-0.036342866718769073
train_tps_Step：3300 | Reonstruction loss is 101.6783218383789 | Norm loss is  0.0 | Total loss is 101.6783218383789 | TPS_var_mean is  0.0 | feature_var_mean is-0.03736303001642227
train_tps_Step：3400 | Reonstruction loss is 100.71012878417969 | Norm loss is  0.0 | Total loss is 100.71012878417969 | TPS_var_mean is  0.0 | feature_var_mean is-0.038258813321590424
train_tps_Step：3500 | Reonstruction loss is 99.76725006103516 | Norm loss is  0.0 | To

...

train_tps_Step：9400 | Reonstruction loss is 78.93475341796875 | Norm loss is  0.0 | Total loss is 78.93475341796875 | TPS_var_mean is  0.0 | feature_var_mean is-0.03197912499308586
train_tps_Step：9500 | Reonstruction loss is 78.80996704101562 | Norm loss is  0.0 | Total loss is 78.80996704101562 | TPS_var_mean is  0.0 | feature_var_mean is-0.031586628407239914
train_tps_Step：9600 | Reonstruction loss is 78.69676208496094 | Norm loss is  0.0 | Total loss is 78.69676208496094 | TPS_var_mean is  0.0 | feature_var_mean is-0.031419701874256134
train_tps_Step：9700 | Reonstruction loss is 78.58818817138672 | Norm loss is  0.0 | Total loss is 78.58818817138672 | TPS_var_mean is  0.0 | feature_var_mean is-0.031207408756017685
train_tps_Step：9800 | Reonstruction loss is 78.48888397216797 | Norm loss is  0.0 | Total loss is 78.48888397216797 | TPS_var_mean is  0.0 | feature_var_mean is-0.030877135694026947
train_tps_Step：9900 | Reonstruction loss is 78.40560150146484 | Norm loss is  0.0 | Total l

train_tps_Step：14000 | Reonstruction loss is 76.64530181884766 | Norm loss is  0.0 | Total loss is 76.64530181884766 | TPS_var_mean is  0.0 | feature_var_mean is-0.01963306963443756
train_tps_Step：14100 | Reonstruction loss is 76.63361358642578 | Norm loss is  0.0 | Total loss is 76.63361358642578 | TPS_var_mean is  0.0 | feature_var_mean is-0.019398938864469528
train_tps_Step：14200 | Reonstruction loss is 76.62093353271484 | Norm loss is  0.0 | Total loss is 76.62093353271484 | TPS_var_mean is  0.0 | feature_var_mean is-0.01930975914001465
train_tps_Step：14300 | Reonstruction loss is 76.60786437988281 | Norm loss is  0.0 | Total loss is 76.60786437988281 | TPS_var_mean is  0.0 | feature_var_mean is-0.019301077350974083
train_tps_Step：14400 | Reonstruction loss is 76.5931396484375 | Norm loss is  0.0 | Total loss is 76.5931396484375 | TPS_var_mean is  0.0 | feature_var_mean is-0.019211839884519577
train_tps_Step：14500 | Reonstruction loss is 76.57713317871094 | Norm loss is  0.0 | Tota

train_tps_Step：3300 | Reonstruction loss is 101.81533813476562 | Norm loss is  0.0 | Total loss is 101.81533813476562 | TPS_var_mean is  0.0 | feature_var_mean is-0.03737271577119827
train_tps_Step：3400 | Reonstruction loss is 100.87918853759766 | Norm loss is  0.0 | Total loss is 100.87918853759766 | TPS_var_mean is  0.0 | feature_var_mean is-0.03809255734086037
train_tps_Step：3500 | Reonstruction loss is 99.98120880126953 | Norm loss is  0.0 | Total loss is 99.98120880126953 | TPS_var_mean is  0.0 | feature_var_mean is-0.037918154150247574
train_tps_Step：3600 | Reonstruction loss is 99.20780944824219 | Norm loss is  0.0 | Total loss is 99.20780944824219 | TPS_var_mean is  0.0 | feature_var_mean is-0.03771030530333519
train_tps_Step：3700 | Reonstruction loss is 98.42282104492188 | Norm loss is  0.0 | Total loss is 98.42282104492188 | TPS_var_mean is  0.0 | feature_var_mean is-0.03666083887219429
train_tps_Step：3800 | Reonstruction loss is 97.64923095703125 | Norm loss is  0.0 | Total 

train_tps_Step：7900 | Reonstruction loss is 81.36502838134766 | Norm loss is  0.0 | Total loss is 81.36502838134766 | TPS_var_mean is  0.0 | feature_var_mean is-0.03652843460440636
train_tps_Step：8000 | Reonstruction loss is 81.2010726928711 | Norm loss is  0.0 | Total loss is 81.2010726928711 | TPS_var_mean is  0.0 | feature_var_mean is-0.036478638648986816
train_tps_Step：8100 | Reonstruction loss is 81.0553970336914 | Norm loss is  0.0 | Total loss is 81.0553970336914 | TPS_var_mean is  0.0 | feature_var_mean is-0.035748764872550964
train_tps_Step：8200 | Reonstruction loss is 80.90121459960938 | Norm loss is  0.0 | Total loss is 80.90121459960938 | TPS_var_mean is  0.0 | feature_var_mean is-0.035189609974622726
train_tps_Step：8300 | Reonstruction loss is 80.75603485107422 | Norm loss is  0.0 | Total loss is 80.75603485107422 | TPS_var_mean is  0.0 | feature_var_mean is-0.03508085012435913
train_tps_Step：8400 | Reonstruction loss is 80.6128158569336 | Norm loss is  0.0 | Total loss is

train_tps_Step：12500 | Reonstruction loss is 77.52528381347656 | Norm loss is  0.0 | Total loss is 77.52528381347656 | TPS_var_mean is  0.0 | feature_var_mean is-0.02304976060986519
train_tps_Step：12600 | Reonstruction loss is 77.49331665039062 | Norm loss is  0.0 | Total loss is 77.49331665039062 | TPS_var_mean is  0.0 | feature_var_mean is-0.022240079939365387
train_tps_Step：12700 | Reonstruction loss is 77.47188568115234 | Norm loss is  0.0 | Total loss is 77.47188568115234 | TPS_var_mean is  0.0 | feature_var_mean is-0.0217027235776186
train_tps_Step：12800 | Reonstruction loss is 77.43881225585938 | Norm loss is  0.0 | Total loss is 77.43881225585938 | TPS_var_mean is  0.0 | feature_var_mean is-0.02161893993616104
train_tps_Step：12900 | Reonstruction loss is 77.41638946533203 | Norm loss is  0.0 | Total loss is 77.41638946533203 | TPS_var_mean is  0.0 | feature_var_mean is-0.021469546481966972
train_tps_Step：13000 | Reonstruction loss is 77.40141296386719 | Norm loss is  0.0 | Tota

In [2]:
import numpy as np
import scipy.misc
import os
#zz = np.load('groundtruth_z/'+'feature_trained'+str(0)+'.npy')#硅胶
zz = np.load('groundtruth_z/'+'feature_trained_invivo'+str(1)+'.npy')#真实
for i in range(2,10):
    z_test = np.load('groundtruth_z/'+'feature_trained_invivo'+str(i)+'.npy')
    zz = np.concatenate((zz,z_test),axis = 0) 
print(zz.shape)
np.save(os.path.join('groundtruth_z/', 'feature_trained_final'+'.npy'), zz)


(450, 30, 1)


In [ ]:



    loss2[:,(128-24):(128+25),(128-24):(128+25),:] = tf.assign(loss2[:,(128-24):(128+25),(128-24):(128+25),:],(tf.constant(np.zeros([sz_params.data_size,(128-24):(128+25),(128-24):(128+25),:3]))))



In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
sess=tf.Session()
a=tf.constant(1)
b=tf.constant(3)
print(sess.run(a+b))

/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/py36/lib/python3.6/site-

4


In [4]:
import tensorflow as tf
import numpy as np
import keras.backend as K
sess=tf.Session()


aa = np.zeros([1,20,20],dtype = np.float32)
compa0 = tf.Variable(aa, dtype=tf.float32,name='contr_val', trainable=False)

compa0 = compa0[:,7:12,7:12].assign(5*np.ones([1,5,5]))

shape = tf.shape(compa0)
jm = tf.fill(shape,3.0)
jm2 = tf.fill(shape,0.0)
jm3 = tf.fill(shape,True)
compa1=tf.greater(compa0,jm2)#############0会做成False的！！！！！
compa2=tf.less(compa0,jm)
compa3=tf.greater(compa0,jm)

pad1 = np.array([[0, 0], [0, 0], [0, 5]])
# tf.pad进行填充
compa3_pad = tf.pad(compa3, pad1, name='pad_1')
compa3_pad2 = tf.logical_not(compa3_pad, name=None)


compa3_1 = tf.slice(compa3_pad2, [0, 0, 1], [-1, 20, 20])
compa3_2 = tf.slice(compa3_pad2, [0, 0, 5], [-1, 20, 20])


compa = tf.to_float(jm3&compa2&compa3_1&compa3_2)


compa4 = tf.expand_dims(compa, -1)  
kernel = tf.zeros([3,3,1],dtype = tf.float32)####22指增加左和上，33上下左右都加1
compa4 = tf.nn.erosion2d(compa4, kernel, strides = [1, 1, 1, 1], rates = [1, 1, 1, 1], padding="SAME")



kernel = tf.zeros([3,3,1],dtype = tf.float32)
compa_smooth = tf.nn.dilation2d(compa4, kernel, strides = [1, 1, 1, 1], rates = [1, 1, 1, 1], padding="SAME")

print(compa_smooth)
jm5 = tf.fill(shape,0.5)
jm5 = tf.expand_dims(jm5, -1)  
compa_smooth = tf.to_float(tf.less(compa_smooth,jm5))

#kernel = tf.zeros([3,3,1],dtype = tf.float32)
#compa_smooth = tf.nn.dilation2d(compa2, kernel, strides = [1, 1, 1, 1], rates = [1, 1, 1, 1], padding="SAME")

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())



a = sess.run(compa0)
print(a)
a = sess.run(compa2)
print(a)
a = sess.run(compa3)
print(a)
a = sess.run(compa3_pad)
print(a)
a = sess.run(compa3_pad2)
print(a)
a = sess.run(compa3_2)
print(a)
a = sess.run(compa)
print(a)
a = sess.run(compa4[:,:,:,0])
print(a)
a = sess.run(compa_smooth[:,:,:,0])
print(a)
a = sess.run(jm5[:,:,:,0])
print(a)


Tensor("Dilation2D_3:0", shape=(1, 20, 20, 1), dtype=float32)
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 5. 5. 5. 5. 5. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 5. 5. 5. 5. 5. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 5. 5. 5. 5. 5. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 5. 5. 5. 5. 5. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 5. 5. 5. 5. 5. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 